In [1]:
import ast
from typing import Annotated, TypedDict

from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_core.vectorstores.in_memory import InMemoryVectorStore
from langchain_ollama import ChatOllama, OllamaEmbeddings

from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

/home/anseh/.pyenv/versions/langchain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
@tool
def calculator(query: str) -> str:
    """계산기. 수식만 입력받습니다."""
    return ast.literal_eval(query)


search = DuckDuckGoSearchRun()
tools = [search, calculator]

embeddings = OllamaEmbeddings(model="nomic-embed-text")
model = ChatOllama(model="granite3.3:2b", temperature=0.1)

tools_retriever = InMemoryVectorStore.from_documents(
    [Document(tool.description, metadata={"name": tool.name}) for tool in tools],
    embeddings,
).as_retriever()

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    selected_tools: list[str]


def model_node(state: State) -> State:
    selected_tools = [tool for tool in tools if tool.name in state["selected_tools"]]
    res = model.bind_tools(selected_tools).invoke(state["messages"])
    return {"messages": res}


def select_tools(state: State) -> State:
    query = state["messages"][-1].content
    tool_docs = tools_retriever.invoke(query)
    return {"selected_tools": [doc.metadata["name"] for doc in tool_docs]}


builder = StateGraph(State)
builder.add_node("select_tools", select_tools)
builder.add_node("model", model_node)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "select_tools")
builder.add_edge("select_tools", "model")
builder.add_conditional_edges("model", tools_condition)
builder.add_edge("tools", "model")

graph = builder.compile()

In [ ]:
# 예시
input = {
    "messages": [HumanMessage("미국의 제30대 대통령이 사망했을 때 몇 살이었나요?")]
}

for c in graph.stream(input):
    print(c)

{'select_tools': {'selected_tools': ['calculator', 'duckduckgo_search']}}
{'model': {'messages': AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite3.3:2b', 'created_at': '2025-11-03T01:11:54.103163549Z', 'done': True, 'done_reason': 'stop', 'total_duration': 10796599596, 'load_duration': 2120369119, 'prompt_eval_count': 262, 'prompt_eval_duration': 6137614421, 'eval_count': 40, 'eval_duration': 2529274280, 'model_name': 'granite3.3:2b'}, id='run--33282864-3660-491f-b926-fb750ef47130-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': '제30대 미국 대통령 사망 후 몇 살'}, 'id': 'e89c8292-c6c8-4f81-84b6-728c5dcb4f33', 'type': 'tool_call'}], usage_metadata={'input_tokens': 262, 'output_tokens': 40, 'total_tokens': 302})}}
{'tools': {'messages': [ToolMessage(content='역대 미국 대통령 에 대해서 명단과 함께 소개하는 문서. 한국과 달리 미국은 연임한 대통령의 경우 전체 임기를 한 대로 센다. 따라서 조지 워싱턴 은 한국의 대수 문화대로 표기하면 제1·2대 대통령이지만 미국의 방식에 따라서 제1대 대통령으로 표기 ... 미국은 2025년 10월 현재 도널드 트럼프가 제47대 대통령으로 재직 중이다. 초대 대통령인 조